## Chapter 14 

# Files
## 14.1 Persistence
Most of the programs we have seen so far are transient in the sense that they run for a short time and produce some output, but when they end, their data disappears. If you run the program again, it starts with a clean slate.
Other programs are persistent: they run for a long time (or all the time); they keep at least some of their data in permanent storage (a hard drive, for example); and if they shut down and restart, they pick up where they left off.
Examples of persistent programs are operating systems, which run pretty much whenever a computer is on, and web servers, which run all the time, waiting for requests to come in on the network.
One of the simplest ways for programs to maintain their data is by reading and writing text files. We have already seen programs that read text files; in this chapter we will see programs that write them.
An alternative is to store the state of the program in a database. In this chapter I will present a simple database and a module, pickle, that makes it easy to store program data.
## 14.2 Reading and writing
A text file is a sequence of characters stored on a permanent medium like a hard drive, flash memory, or CD-ROM. We saw how to open and read a file in Section 9.1.
To write a file, you have to open it with mode 'w' as a second parameter:

In [ ]:
>>> fout = open('output.txt', 'w')
>>> print (fout)

If the file already exists, opening it in write mode clears out the old data and starts fresh, so be careful! If the file doesn’t exist, a new one is created.
The write method puts data into the file.

In [ ]:
>>> line1 = "This here's the wattle,\n"
>>> fout.write(line1)

In [ ]:
# Again, the file object keeps track of where it is, so if you call write again, it adds the new data to the end.
>>> line2 = "the emblem of our land.\n"
>>> fout.write(line2)
# When you are done writing, you have to close the file.
>>> fout.close()

## 14.3 Format operator
The argument of write has to be a string, so if we want to put other values in a file, we have to convert them to strings. The easiest way to do that is with str:

In [ ]:
>>> x = 52
>>> fout.write(str(x))

An alternative is to use the format operator, %. When applied to integers, % is the modulus operator. But when the first operand is a string, % is the format operator.
The first operand is the format string, which contains one or more format sequences, which specify how the second operand is formatted. The result is a string.
For example, the format sequence '%d' means that the second operand should be format- ted as an integer (d stands for “decimal”):

In [ ]:
>>> camels = 42
>>> '%d' % camels
'42'

The result is the string '42', which is not to be confused with the integer value 42.
A format sequence can appear anywhere in the string, so you can embed a value in a
sentence:

In [ ]:
>>> camels = 42
>>> 'I have spotted %d camels.' % camels
'I have spotted 42 camels.'

If there is more than one format sequence in the string, the second argument has to be a tuple. Each format sequence is matched with an element of the tuple, in order.
The following example uses '%d' to format an integer, '%g' to format a floating-point num- ber (don’t ask why), and '%s' to format a string:

In [ ]:
>>> 'In %d years I have spotted %g %s.' % (3, 0.1, 'camels')
'In 3 years I have spotted 0.1 camels.'
The number of elements in the tuple has to match the number of format sequences in the string. Also, the types of the elements have to match the format sequences:
>>> '%d %d %d' % (1, 2)
TypeError: not enough arguments for format string
>>> '%d' % 'dollars'
TypeError: illegal argument type for built-in operation

### 14.4. Filenames and paths
 In the first example, there aren’t enough elements; in the second, the element is the wrong type.
The format operator is powerful, but it can be difficult to use. You can read more about it at http://docs.python.org/2/library/stdtypes.html#string-formatting.
Files are organized into directories (also called “folders”). Every running program has a “current directory,” which is the default directory for most operations. For example, when you open a file for reading, Python looks for it in the current directory.
The os module provides functions for working with files and directories (“os” stands for “operating system”). os.getcwd returns the name of the current directory:

In [ ]:
>>> import os
>>> cwd = os.getcwd()
>>> print(cwd)

cwd stands for “current working directory.” The result in this example is /home/dinsdale, which is the home directory of a user named dinsdale.
A string like cwd that identifies a file is called a path. A relative path starts from the current directory; an absolute path starts from the topmost directory in the file system.
The paths we have seen so far are simple filenames, so they are relative to the current directory. To find the absolute path to a file, you can use os.path.abspath:

In [ ]:
>>> os.path.abspath('memo.txt')
# '/home/dinsdale/memo.txt'

os.path.exists checks whether a file or directory exists: 

In [ ]:
>>> os.path.exists('memo.txt')

In [ ]:
If it exists, os.path.isdir checks whether it’s a directory: 

In [ ]:
>>> os.path.exists('memo.txt')
True
If it exists, os.path.isdir checks whether it’s a directory: 
>>> os.path.isdir('memo.txt')
False
>>> os.path.isdir('music')
True
Similarly, os.path.isfile checks whether it’s a file.
os.listdir returns a list of the files (and other directories) in the given directory:
>>> os.listdir(cwd)
['music', 'photos', 'memo.txt']

To demonstrate these functions, the following example “walks” through a directory, prints the names of all the files, and calls itself recursively on all the directories.

In [ ]:
def walk(dirname):
    for name in os.listdir(dirname):
        path = os.path.join(dirname, name)
        if os.path.isfile(path):
            print(path)
        else:
            walk(path)

In [ ]:
# walk(os.getcwd())

os.path.join takes a directory and a file name and joins them into a complete path. 

### Exercise 14.1. 
The os module provides a function called walk that is similar to this one but more versatile. Read the documentation and use it to print the names of the files in a given directory and its subdirectories.

In [ ]:
import os
di=os.getcwd()
for top,dire,files in os.walk(di):
     for filename in files:
#             print (top)
            print (os.path.join(top,filename))


Solution: http://greenteapress.com/thinkpython/code/walk.py .

### 14.5 Catching exceptions
A lot of things can go wrong when you try to read and write files. If you try to open a file that doesn’t exist, you get an IOError:

In [ ]:
>>> fin = open('bad_file')
IOError: [Errno 2] No such file or directory: 'bad_file'
If you don’t have permission to access a file:
>>> fout = open('/etc/passwd', 'w')
IOError: [Errno 13] Permission denied: '/etc/passwd'
And if you try to open a directory for reading, you get
>>> fin = open('/home')
IOError: [Errno 21] Is a directory

To avoid these errors, you could use functions like os.path.exists and os.path.isfile, but it would take a lot of time and code to check all the possibilities (if “Errno 21” is any indication, there are at least 21 things that can go wrong)m.
It is better to go ahead and try and deal with problems if they happen—which is exactly what the try statement does. The syntax is similar to an if statement:

In [ ]:
try:
    fin = open('bad_file')
    for line in fin:
        print (line)
    fin.close()
except:
    print ('Something went wrong.')

Python starts by executing the try clause. If all goes well, it skips the except clause and proceeds. If an exception occurs, it jumps out of the try clause and executes the except clause.
Handling an exception with a try statement is called catching an exception. In this example, the except clause prints an error message that is not very helpful. In general, catching an exception gives you a chance to fix the problem, or try again, or at least end the program gracefully.
### Exercise 14.2. 
Write a function called sed that takes as arguments a pattern string, a replacement string, and two filenames; it should read the first file and write the contents into the second file (creating it if necessary). If the pattern string appears anywhere in the file, it should be replaced with the replacement string.
If an error occurs while opening, reading, writing or closing files, your program should catch the exception, print an error message, and exit. 
Solution: http://greenteapress.com/thinkpython/code/sed.py.

### 14.6 Databases
A database is a file that is organized for storing data. Most databases are organized like a dictionary in the sense that they map from keys to values. The biggest difference is that the database is on disk (or other permanent storage), so it persists after the program ends.
The module anydbm provides an interface for creating and updating database files. As an example, I’ll create a database that contains captions for image files.
Opening a database is similar to opening other files:

In [ ]:
>>> import dbm
>>> db = dbm.open('captions', 'c')

In [ ]:
db.close()

In [ ]:
>>> db['cleese.png'] = 'Photo of John Cleese.'

In [ ]:
>>> print (db['cleese.png'])

In [ ]:
>>> db['cleese.png'] = 'Photo of John Cleese doing a silly walk.'
>>> print (db['cleese.png'])

Many dictionary methods, like keys and items, also work with database objects. So does iteration with a for statement.
for key in db:
print key
As with other files, you should close the database when you are done:

In [ ]:
>>> db.close()

### 14.7 Pickling
A limitation of anydbm is that the keys and values have to be strings. If you try to use any other type, you get an error.
The pickle module can help. It translates almost any type of object into a string suitable for storage in a database, and then translates strings back into objects.
pickle.dumps takes an object as a parameter and returns a string representation (dumps is short for “dump string”):

### Exercise 14.3.
If you download my solution to Exercise 12.4 from http://greenteapress.com/thinkpython/code/anagram_sets.py , you’ll see that it creates a dictionary that maps from a sorted string of letters to the list of words that can be spelled with those letters. For example, 'opst' maps to the list ['opts', 'post', 'pots', 'spot', 'stop', 'tops'].
Write a module that imports anagram_sets and provides two new functions: store_anagrams should store the anagram dictionary in a “shelf;” read_anagrams should look up a word and return a list of its anagrams. Solution: http://greenteapress.com/thinkpython/code/anagram_db.py

In [104]:
def signature(s):
    """Returns the signature of this string, which is a string
    that contains all of the letters in order.
    """
    t = list(s)
    t.sort()
    t = ''.join(t)
    return t


def all_anagrams(filename):
    """Finds all anagrams in a list of words.

    filename: string filename of the word list

    Returns: a map from each word to a list of its anagrams.
    """
    d = {}
    for line in open(filename):
        word = line.strip().lower()
        t = signature(word)

        if t not in d:
            d[t] = [word]
        else:
            d[t].append(word)
    return d

In [105]:
import shelve

In [106]:
ag=shelve.open('Files/all_anagrams', 'c')

In [107]:
aa=all_anagrams('Files/words.txt')

In [ ]:
# for key,value in aa.items():
#     ag[key]=str(value)

In [109]:
ag[signature('eel')]

['eel', 'lee']

### 14.8 Pipes
Most operating systems provide a command-line interface, also known as a shell. Shells usually provide commands to navigate the file system and launch applications. For example, in Unix you can change directories with cd, display the contents of a directory with ls, and launch a web browser by typing (for example) firefox.
Any program that you can launch from the shell can also be launched from Python using a pipe. A pipe is an object that represents a running program.
For example, the Unix command ls -l normally displays the contents of the current directory (in long format). You can launch ls with os.popen : 

In [ ]:
import os
>>> cmd = 'cd'
>>> fp = os.popen(cmd)
for i in fp:
    print (i)
    fp.close()

The argument is a string that contains a shell command. The return value is an object that behaves just like an open file. You can read the output from the ls process one line at a time with readline or get the whole thing at once with read:
popen is deprecated now, which means we are supposed to stop using it and start using the subprocess module. But for simple cases, I find subprocess more complicated than necessary. So I am going to keep using popen until they take it away.

### Exercise 14.4. 

In a large collection of MP3 files, there may be more than one copy of the same song, stored in different directories or with different file names. The goal of this exercise is to search for duplicates.
1. Write a program that searches a directory and allofits subdirectories,recursively,and returns a list of complete paths for all files with a given suffix (like .mp3). Hint: os.path provides several useful functions for manipulating file and path names.
2. To recognize duplicates, you can use md5sum to compute a “checksum” for each files. If two files have the same checksum, they probably have the same contents.
3. To double-check, you can use the Unix command diff.
Solution: http://greenteapress.com/thinkpython/code/find_duplicates.py.

In [98]:
cwd=os.getcwd()

In [55]:
import os
cmd='ls '+repr(cwd)
print(cmd) 

ls /Users/danielfriedman/Documents/Git/Think\ Python/Files/diff


In [56]:
df=os.popen(cmd)
res=df.read()
print(res)
stat=df.close()

file1 copy.txt
file1.txt
file2.txt



In [57]:
def rpath(path):
#     escape the whitespace with "\"
    return path.replace(' ','\ ')

In [53]:
import os
#initiate list of files with required extentions
filelist=[]
# cwd=os.getcwd()
extension='.txt'
for path,folder,files in os.walk(cwd):

    for file in files:
        name,extension=os.path.splitext(file)
        if extension=='.txt':            
            t=os.path.join(path,file)
            filelist.append(t)

In [77]:
def checksum(file):    
    checksum=os.popen('md5 '+repr(file)) #Use repr to deal with whitespace in file path.
    t=checksum.read().split()[-1] #Get
    stat=checksum.close()
    stat
    return t 

In [87]:
    for stab in filelist:
        if file==stab:
            pass
        else:
            if in_question==checksum(stab):
                print ('File:\n')
                print (file+'\n')
                print ("is duplicate of: \n")
                print (stab)

            

File:

/Users/danielfriedman/Documents/Git/Think Python/Files/diff/file1 copy.txt

is duplicate of: 

/Users/danielfriedman/Documents/Git/Think Python/Files/diff/file1.txt


### 14.9 Writing modules
Any file that contains Python code can be imported as a module. For example, suppose you have a file named wc.py with the following code:

In [96]:
def linecount(filename):
    count = 0
    for line in open(filename):
        count += 1
    return count
print (linecount('wc.py'))

UnsupportedOperation: not readable

### 14.12 Exercises
Exercise 14.6. The urllib module provides methods for manipulating URLs and downloading information from the web. The following example downloads and prints a secret message from thinkpython.com:

In [102]:
import urllib
conn = urllib.request.urlopen('http://greenteapress.com/thinkpython/secret.html')
for line in conn:
    print (line.strip())

b'<title>Secret exercise</title>'
b''
b'<h1>Secret Think Python Exercise</h3>'
b''
b'<p>'
b'If you are reading this, you are probably working on the urllib'
b'exercise from Think Python.'
b''
b'<p>'
b'Next, you should read the documentation of the urllib module at'
b'http://docs.python.org/lib/module-urllib.html'
b''
b'<p>'
b'Then go to www.uszip.com, which provides information about every zip code'
b'in the country.  For example, the URL'
b''
b'<p>'
b'http://www.uszip.com/zip/02492'
b''
b'<p>'
b'provides information about Needham MA, including population, longitude'
b'and latitude, etc.'
b''
b'<p>'
b'Write a program that prompts the user for a zip code and prints the'
b'name and population of the corresponding town.'
b''
b'<p>'
b'Note: the text you get from uszip.com is in HTML, the language most'
b"web pages are written in.  Even if you don't know HTML, you should be"
b'able to extract the information you are looking for.'
b''
b'<p>'
b'By the way, your program is an example of a "scr

Run this code and follow the instructions you see there. 
Solution: http://greenteapress.com/thinkpython/code/zip_code.py .